In [1]:
import pandas as pd
import numpy as np

tweet_df = pd.read_pickle('../data/agg_trans_tweets.pickle.gz', compression='gzip')
tweet_df['i'] = list(range(len(tweet_df)))
tweet_df.set_index('date', inplace=True)
tweet_df

,tweet,author,id,i
date,,,,
2021-04-27 21:45:50,@Alicoh1 @qwosl @kdqd3 @HotlineJosh But strang...,Platonista1,1387161089121701892,0
2021-04-27 21:45:49,RT @DavidPaisley: Not only do we export our tr...,daniellismore,1387161086479241218,1
2021-04-27 21:45:45,RT @vickiringer: NC’s bathroom bill discrimina...,CherieAnne,1387161067000840198,2
2021-04-27 21:45:40,@yelloweycat @Mushr00mSteve @BaddestJeanette A...,encyclopath,1387161048352841728,3
2021-04-27 21:45:40,@thehill This CEO is fired for belittling a gu...,rynob11,1387161046612336641,4
...,...,...,...,...
2021-04-20 08:31:11,@idkwtwhnid @GhostlyWeevil @GayestFesh @bigges...,natleah_,1384424393124761600,303843
2021-04-20 08:30:31,RT @HANNAHLUVSDSGC: north carolina wants to pa...,taoIings,1384424225113624578,303844
2021-04-20 08:30:31,RT @mcclure111: If you live in a country where...,Angstlers1,1384424224715067394,303845


## LDA Performance

How well does LDA perform?

In [2]:
from datetime import datetime

sample_period_df = tweet_df[(datetime(2021, 4, 22) < tweet_df.index) & (tweet_df.index < datetime(2021, 4, 23))]
sample_period_df

,tweet,author,id,i
date,,,,
2021-04-22 23:59:57,RT @jasondogwood: Remember Kai Shappley the 10...,MonaVieBeachBum,1385382900623822849,98098
2021-04-22 23:59:53,RT @CenterOnBudget: 🧵: Today Secretary Fudge a...,karla_handley,1385382886333898752,98099
2021-04-22 23:59:48,@bokuroos DORRY IM SO EXCITED LOOK https://t.c...,puppysfae,1385382861692354560,98100
2021-04-22 23:59:37,RT @ACLU: The DOJ then announced today that de...,TheOmegaVirus,1385382815869505540,98101
2021-04-22 23:59:28,@TrueColorsUnite @SecFudge I would be interest...,TaylorW09986445,1385382779651821572,98102
...,...,...,...,...
2021-04-22 00:00:11,RT @rauhling_bizzle: A reminder: Support BLACK...,blackheartavery,1385020572661436416,255580
2021-04-22 00:00:08,RT @K_IngalaSmith: I have to get this off my c...,Mykiwitwit,1385020559772196866,255581
2021-04-22 00:00:05,RT @morganholleb: here's a list of trans and q...,Double4Designer,1385020548065873920,255582


In [3]:
import gensim
from nltk import TweetTokenizer, WordNetLemmatizer
import sys
import os
sys.path.append("..")
import utils.tokens as ut

TFIDF_MODEL_PATH = '../models/d2d-tfidf.model'
stopwords = ut.build_stopword_set()
documents = list(ut.tokenize_tweets(tweet_df['tweet'], stopwords))
dictionary = gensim.corpora.Dictionary(documents)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

/home/astrid/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package stopwords to /home/astrid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/astrid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
if os.path.exists(TFIDF_MODEL_PATH):
    tfidf = gensim.models.TfidfModel.load(TFIDF_MODEL_PATH)
else:
    tfidf = gensim.models.TfidfModel(bow_corpus)
    tfidf.save(TFIDF_MODEL_PATH)
corpus_tfidf = tfidf[bow_corpus]
corpus_tfidf

In [10]:

def train_lda(store_path, sample, n_topics, passes=20):
    if os.path.exists(store_path):
        lda_model_tfidf = gensim.models.LdaMulticore.load(store_path)
    else:
        lda_model_tfidf = gensim.models.LdaMulticore(
            sample,
            num_topics=n_topics,
            id2word=dictionary,
            alpha='asymmetric',
            passes=passes,
            workers=16
        )
        lda_model_tfidf.save(store_path)
    return lda_model_tfidf

sample_tfidf = [corpus_tfidf[i] for i in sample_period_df['i']]
sample_lda = train_lda('../models/d2d-sample-lda.model', corpus_tfidf, 20)
sample_lda

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/astrid/anaconda3/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [11]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(sample_lda, bow_corpus, dictionary)

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.380378 -0.192448       1        1  42.144653
0     -0.328331 -0.165309       2        1  28.498318
2     -0.285028 -0.034624       3        1   6.458543
3     -0.209498  0.157254       4        1   5.122684
5     -0.114934  0.253071       5        1   4.116288
4     -0.181164  0.070225       6        1   3.571842
7     -0.076308  0.102930       7        1   2.987474
6     -0.015916  0.182445       8        1   2.362356
8      0.095139  0.025969       9        1   1.002470
10     0.125445 -0.043991      10        1   0.576688
9      0.115447 -0.030084      11        1   0.575846
11     0.130658 -0.042346      12        1   0.419651
14     0.132357 -0.032718      13        1   0.394994
15     0.138319 -0.034900      14        1   0.308499
12     0.143182 -0.035392      15        1   0.288132
13     0.146276 -0.035390      16        1   0.281828
16     0.132964 -0.040253      17        1   0.249559
19     0.143245 -0.037363      18        1   0.216346
18     0.144592 -0.033669      19        1   0.213185
17     0.143933 -0.033408      20        1   0.210644, topic_info=                 Term          Freq         Total Category  logprob  loglift
32               bill  19027.000000  19027.000000  Default  30.0000  30.0000
17        transgender  28202.000000  28202.000000  Default  29.0000  29.0000
766            signed  10444.000000  10444.000000  Default  28.0000  28.0000
141             woman  14539.000000  14539.000000  Default  27.0000  27.0000
172              also   9459.000000   9459.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
9566  marginalsierede      0.122214      4.083611  Topic20 -11.0805   2.6538
9554           binære      0.122214      4.083611  Topic20 -11.0805   2.6538
9573   statistikkerne      0.122214      4.083611  Topic20 -11.0805   2.6538
3794               er      1.482466     66.525529  Topic20  -8.5848   2.3589
1930        degrading      0.149572     93.753912  Topic20 -10.8785  -0.2779

[1025 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
14668     14  0.782559  #antimasker
18261     14  0.734291  #antivaxxer
15574      4  0.272913   #apr25coup
15574      5  0.725497   #apr25coup
1169       5  0.998161   #apr27coup
...      ...       ...          ...
2186      18  0.330275     𝘼𝙇𝙇𝙄𝘼𝙉𝘾𝙀
2187      18  0.330275          𝙇𝙂𝘽
2188      18  0.330275        𝙋𝙍𝙄𝘿𝙀
2189      18  0.330275    𝙎𝙏𝘼𝙏𝙀𝙈𝙀𝙉𝙏
2190      18  0.330275        𝙒𝙊𝙇𝘿𝙎

[1833 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4, 6, 5, 8, 7, 9, 11, 10, 12, 15, 16, 13, 14, 17, 20, 19, 18])

In [ ]:
import sklearn

def tokenizer(tweet):
    return ut.tokenize_tweet(tweet, stopwords)

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(tokenizer=tokenizer)
vectorizer.fit(tweet_df['tweet'])
#sklearn.decomposition.LatentDirichletAllocation()

/home/astrid/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/astrid/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
